In [1]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
def clear_cuda_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("CUDA cache cleared and memory freed.")


In [3]:
import json
import os
import re
from datasets import Dataset, DatasetDict
from transformers import (
    T5Tokenizer,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    T5ForConditionalGeneration, 
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
import nltk
from typing import List, Tuple
from nltk.tokenize import sent_tokenize
from datasets import Dataset, concatenate_datasets
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
import torch
from sklearn.model_selection import train_test_split

2025-04-06 01:25:46.548464: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 01:25:46.565218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743902746.585703    7321 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743902746.591963    7321 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-06 01:25:46.613267: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Training code

In [4]:
!pip install accelerate -U

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
!pip install evaluate nltk rouge_score bert_score transformers[torch]

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
## data preparation and making decisions for additional tokens

In [6]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
print(tokenizer.tokenize("brick:Natural_Gas rdfs:label ?label SELECT WHERE brick:Air_Flow_Sensor"))
# Output: ['brick', ':', 'Natural', '_', 'Gas', 'rdfs', ':', 'label', '?', 'label']

['▁brick', ':', 'N', 'atura', 'l', '_', 'G', 'a', 's', '▁', 'r', 'd', 'f', 's', ':', 'l', 'abel', '▁', '?', 'l', 'abel', '▁', 'SEL', 'ECT', '▁W', 'HER', 'E', '▁brick', ':', 'Air', '_', 'Flow', '_', 'S', 'en', 's', 'or']


In [12]:
import json

# Load the first JSON file
with open("updated_bldg_question_pairs_entities.json", "r") as f1:
    data1 = json.load(f1)

# Load the second JSON file
with open("updated_combined_output_with_entity.json", "r") as f2:
    data2 = json.load(f2)

# Function to filter and extract only the required keys
def filter_entry(entry):
    return {
        "question": entry["question"],
        "entity": entry["entity"],
        "sparql": entry["sparql"]
    }

# Process both datasets and combine them
merged_data = []

# Filter entries from file1.json
for entry in data1:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Filter entries from file2.json
for entry in data2:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Save the merged dataset to a new JSON file
with open("merged_output1.json", "w") as f:
    json.dump(merged_data, f, indent=4)

print("Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'.")

Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'.


In [13]:
import json

# Load the first JSON file
with open("abacws_bldg_question_pairs_entities.json", "r") as f1:
    data1 = json.load(f1)

# Load the second JSON file
with open("abacws_bldg_timeseries_question_pairs_entities.json", "r") as f2:
    data2 = json.load(f2)

 # Load the second JSON file
with open("updated_bldg_question_pairs_entities.json", "r") as f3:
    data3 = json.load(f3)


# Function to filter and extract only the required keys
def filter_entry(entry):
    return {
        "question": entry["question"],
        "entity": entry["entity"],
        "sparql": entry["sparql"]
    }

# Process both datasets and combine them
merged_data = []

# Filter entries from file1.json
for entry in data1:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Filter entries from file2.json
for entry in data2:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Filter entries from file2.json
for entry in data3:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Save the merged dataset to a new JSON file
with open("merged_output2.json", "w") as f:
    json.dump(merged_data, f, indent=4)

print("Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'.")

Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'.


In [27]:
import json

# Load the first JSON file
with open("merged_output1.json", "r") as f1:
    data1 = json.load(f1)

# Load the second JSON file
with open("merged_output2.json", "r") as f2:
    data2 = json.load(f2)

# Function to filter and extract only the required keys
def filter_entry(entry):
    return {
        "question": entry["question"],
        "entity": entry["entity"],
        "sparql": entry["sparql"]
    }

# Process both datasets and combine them
merged_data = []

# Filter entries from file1.json
for entry in data1:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Filter entries from file2.json
for entry in data2:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Save the merged dataset to a new JSON file
with open("training_data.json", "w") as f:
    json.dump(merged_data, f, indent=4)

print("Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'.")

Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'.


In [28]:
import json
import random

# Function to read names from a file (one per line)
def load_names_from_file(filename):
    with open(filename, "r") as f:
        # Strip whitespace and filter out empty lines
        names = [line.strip() for line in f if line.strip()]
    return names

# Load sensors and zones from files
sensors = load_names_from_file("brick_sensors.txt")
zones = load_names_from_file("bldg_zones.txt")

# Add 'brick:' prefix to sensors and 'bldg:' prefix to zones if not already present
sensors = [f"brick:{s}" if not s.startswith("brick:") else s for s in sensors]
zones = [f"bldg:{z}" if not z.startswith("bldg:") else z for z in zones]

# Function to generate SPARQL queries and dataset entries
def generate_sparql_dataset(sensors_list, zones_list, num_variations=1000):
    dataset = []
    
    # Base entity template (e.g., "bldg:<Zone> \n brick:<SensorName>")
    def get_entity(sensor, zone):
        sensor_name = sensor.split(":")[1]  # Extract sensor name after "brick:"
        zone_name = zone.split(":")[1]      # Extract zone name after "bldg:"
        return f"bldg:{zone_name} \n brick:{sensor_name}"  # Newline-separated format
    
    for _ in range(num_variations):
        # Randomly select a sensor and zone for each variation
        selected_sensor = random.choice(sensors_list)
        selected_zone = random.choice(zones_list)
        random_id = random.randint(1, 9999)  # Still needed for question2 if uncommented
        selected_entity = get_entity(selected_sensor, selected_zone)
        
        # Variation 1: General question about sensor in a zone
        question1 = f"Tell me the name or label  of the {selected_sensor.split(':')[1].replace('_', ' ').lower()} in the {selected_zone.split(':')[1].replace('-', ' ').replace('_', ' ')}."
        sparql1 = f"SELECT ?label WHERE {{ ?sensor a {selected_sensor} ; brick:hasLocation {selected_zone} ; rdfs:label ?label . }}"
        entry1 = {
            "question": question1,
            "entity": selected_entity,
            "sparql": sparql1
        }
        dataset.append(entry1)
        
        # Variation 2: Specific question about the sensor entity (commented out)
        # Note: If uncommented, this would need adjustment since entity no longer includes random_id
        # question2 = f"Tell me the label of {selected_sensor.split(':')[1]}_{random_id}."
        # sparql2 = f"SELECT ?label WHERE {{ {selected_entity} a {selected_sensor} ; rdfs:label ?label . }}"
        # entry2 = {
        #     "question": question2,
        #     "entity": selected_entity,
        #     "sparql": sparql2
        # }
        # dataset.append(entry2)
    
    return dataset

# Generate a huge dataset with 1000 variations (1000 total entries since Variation 2 is commented)
dataset = generate_sparql_dataset(sensors, zones, num_variations=5000)

# Save to JSON file
with open("sparql_dataset1.json", "w") as f:
    json.dump(dataset, f, indent=4)

print("Huge dataset generated and saved to 'huge_sparql_dataset.json'.")

Huge dataset generated and saved to 'huge_sparql_dataset.json'.


In [ ]:
## v2 adding curly braces

In [1]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
def clear_cuda_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("CUDA cache cleared and memory freed.")


import os
os.environ["WANDB_MODE"] = "offline"

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
import gc
import json
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import evaluate
import torch
import nltk
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq

# Set CUDA device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
nltk.download("punkt", quiet=True)

# 1. LOAD AND PREPARE DATA
DATA_FILE = "training_data.json"
with open(DATA_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"Loaded {len(data)} records from {DATA_FILE}")
print("Example record:", data[0])

# Create multi-task training pairs
inputs, targets = [], []
for record in data:
    question = record.get("question", "")
    entity = record.get("entity", "")
    sparql = record.get("sparql", "")
#     response = record.get("sparql_response", "")
#     explanation = record.get("explanation", "")
    
    # Task 1: NL to SPARQL
    if question and entity and sparql:
        inputs.append(f"task: generate_sparql\ninput: {question}\nentity{entity}")
        targets.append(sparql)
    
#     # Task 2: Summarize response
#     if question and response and explanation:
#         inputs.append(f"task: summarize_response\nquestion: {question}\nresponse: {response}")
#         targets.append(explanation)

print(f"Generated {len(inputs)} total training pairs from {len(data)} records.")
train_inputs, val_inputs, train_targets, val_targets = train_test_split(
    inputs, targets, test_size=0.1, random_state=42
)
print(f"Train size: {len(train_inputs)} | Validation size: {len(val_inputs)}")

# Save splits
train_data = [{"input_text": inp, "target_text": tgt} for inp, tgt in zip(train_inputs, train_targets)]
val_data = [{"input_text": inp, "target_text": tgt} for inp, tgt in zip(val_inputs, val_targets)]
with open("train_data_April.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)
with open("val_data_April.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)
print("Saved train_data_April.json and val_data_April.json!")

# Build datasets
raw_datasets = DatasetDict({
    "train": Dataset.from_dict({"input_text": train_inputs, "target_text": train_targets}),
    "validation": Dataset.from_dict({"input_text": val_inputs, "target_text": val_targets})
})
print("Train sample:", raw_datasets["train"][3])
print("Validation sample:", raw_datasets["validation"][3])

# 2. LOAD MODEL & TOKENIZER
model_name = "./v2/checkpoint-9697"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Add custom tokens once

new_tokens = ["{", "}"]
tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))


# custom_tokens = []
# with open("all_relations_and_classes.txt", "r", encoding="utf-8") as f:
#     custom_tokens.extend([line.strip() for line in f.readlines()])
# with open("output_entities.txt", "r", encoding="utf-8") as f:
#     custom_tokens.extend([line.strip() for line in f.readlines()])
# num_added_tokens = tokenizer.add_tokens(custom_tokens)
# model.resize_token_embeddings(len(tokenizer))
# print(f"Added {num_added_tokens} new tokens to the tokenizer!")

# 3. PREPROCESSING
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        text_target=examples["target_text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 4. EVALUATION METRICS
metric_rouge = evaluate.load("rouge")
metric_bleu = evaluate.load("bleu")
metric_meteor = evaluate.load("meteor")
metric_bertscore = evaluate.load("bertscore")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    
    results = {}
    rouge_result = metric_rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    results.update(rouge_result)
    bleu_result = metric_bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    results["bleu"] = bleu_result["bleu"]
    meteor_result = metric_meteor.compute(predictions=decoded_preds, references=decoded_labels)
    results["meteor"] = meteor_result["meteor"]
    bertscore_result = metric_bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    results["bertscore_precision"] = np.mean(bertscore_result["precision"])
    results["bertscore_recall"] = np.mean(bertscore_result["recall"])
    results["bertscore_f1"] = np.mean(bertscore_result["f1"])
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    results["gen_len"] = np.mean(prediction_lens)
    return results

# 5. TRAINING ARGUMENTS
training_args = Seq2SeqTrainingArguments(
    output_dir="./v3",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    logging_strategy="epoch",
    logging_dir="./v3",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust based on GPU
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=50,
    predict_with_generate=True,
    fp16=True,  # Enable if GPU supports
    report_to=["tensorboard"],
    warmup_steps=500,
    lr_scheduler_type="cosine",
)

# 6. TRAINER SETUP
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 7. TRAIN
trainer.train()
print('training completed')


Loaded 87963 records from training_data.json
Example record: {'question': 'What is the average cooling command reading for AHU 01 over the last hour?', 'entity': 'bldg:bldg1.AHU.AHU01.CCV', 'sparql': 'SELECT ?timeseriesId WHERE { bldg:bldg1.AHU.AHU01.CCV ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId . }'}
Generated 86193 total training pairs from 87963 records.
Train size: 77573 | Validation size: 8620
Saved train_data_April.json and val_data_April.json!
Train sample: {'input_text': 'task: generate_sparql\ninput: Tell me the name or label  of the unoccupied cooling temperature deadband setpoint in the Classroom.\nentitybldg:Classroom \n brick:Unoccupied_Cooling_Temperature_Deadband_Setpoint', 'target_text': 'SELECT ?label WHERE { ?sensor a brick:Unoccupied_Cooling_Temperature_Deadband_Setpoint ; brick:hasLocation bldg:Classroom ; rdfs:label ?label . }'}
Validation sample: {'input_text': 'task: generate_sparql\ninput: Where is the LPG

Map:   0%|          | 0/77573 [00:00<?, ? examples/s]

Map:   0%|          | 0/8620 [00:00<?, ? examples/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Meteor,Bertscore Precision,Bertscore Recall,Bertscore F1,Gen Len
1,0.001700,0.000793,0.573479,0.524517,0.573243,0.573110,0.142844,0.396224,0.931790,0.862826,0.895777,19.000000
2,0.001000,0.000572,0.573878,0.525238,0.573667,0.573513,0.143019,0.396485,0.931852,0.862830,0.895806,18.998724
3,0.001000,0.000595,0.573435,0.524664,0.573205,0.573081,0.142882,0.396276,0.931775,0.862785,0.895746,18.998724
4,0.001000,0.000576,0.573516,0.524738,0.573246,0.573123,0.142883,0.396384,0.931792,0.862784,0.895754,18.998724
5,0.001000,0.000576,0.573416,0.524679,0.573137,0.573026,0.142852,0.396307,0.931767,0.862767,0.895733,18.998724


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to 

training completed


In [ ]:
## V3

In [1]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
def clear_cuda_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("CUDA cache cleared and memory freed.")


import os
os.environ["WANDB_MODE"] = "offline"

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
import gc
import json
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import evaluate
import torch
import nltk
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq

# Set CUDA device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
nltk.download("punkt", quiet=True)

# 1. LOAD AND PREPARE DATA
DATA_FILE = "training_data.json"
with open(DATA_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"Loaded {len(data)} records from {DATA_FILE}")
print("Example record:", data[0])

# Create multi-task training pairs
inputs, targets = [], []
for record in data:
    question = record.get("question", "")
    entity = record.get("entity", "")
    sparql = record.get("sparql", "")
#     response = record.get("sparql_response", "")
#     explanation = record.get("explanation", "")
    
    # Task 1: NL to SPARQL
    if question and entity and sparql:
        inputs.append(f"task: generate_sparql\ninput: {question}\nentity{entity}")
        targets.append(sparql)
    
#     # Task 2: Summarize response
#     if question and response and explanation:
#         inputs.append(f"task: summarize_response\nquestion: {question}\nresponse: {response}")
#         targets.append(explanation)

print(f"Generated {len(inputs)} total training pairs from {len(data)} records.")
train_inputs, val_inputs, train_targets, val_targets = train_test_split(
    inputs, targets, test_size=0.1, random_state=42
)
print(f"Train size: {len(train_inputs)} | Validation size: {len(val_inputs)}")

# Save splits
train_data = [{"input_text": inp, "target_text": tgt} for inp, tgt in zip(train_inputs, train_targets)]
val_data = [{"input_text": inp, "target_text": tgt} for inp, tgt in zip(val_inputs, val_targets)]
with open("train_data_April.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)
with open("val_data_April.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)
print("Saved train_data_April.json and val_data_April.json!")

# Build datasets
raw_datasets = DatasetDict({
    "train": Dataset.from_dict({"input_text": train_inputs, "target_text": train_targets}),
    "validation": Dataset.from_dict({"input_text": val_inputs, "target_text": val_targets})
})
print("Train sample:", raw_datasets["train"][3])
print("Validation sample:", raw_datasets["validation"][3])

# 2. LOAD MODEL & TOKENIZER
model_name = "./v2/checkpoint-48485"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Add custom tokens once

new_tokens = ["{", "}"]
tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))


# custom_tokens = []
# with open("all_relations_and_classes.txt", "r", encoding="utf-8") as f:
#     custom_tokens.extend([line.strip() for line in f.readlines()])
# with open("output_entities.txt", "r", encoding="utf-8") as f:
#     custom_tokens.extend([line.strip() for line in f.readlines()])
# num_added_tokens = tokenizer.add_tokens(custom_tokens)
# model.resize_token_embeddings(len(tokenizer))
# print(f"Added {num_added_tokens} new tokens to the tokenizer!")

# 3. PREPROCESSING
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        text_target=examples["target_text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 4. EVALUATION METRICS
metric_rouge = evaluate.load("rouge")
metric_bleu = evaluate.load("bleu")
metric_meteor = evaluate.load("meteor")
metric_bertscore = evaluate.load("bertscore")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    
    results = {}
    rouge_result = metric_rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    results.update(rouge_result)
    bleu_result = metric_bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    results["bleu"] = bleu_result["bleu"]
    meteor_result = metric_meteor.compute(predictions=decoded_preds, references=decoded_labels)
    results["meteor"] = meteor_result["meteor"]
    bertscore_result = metric_bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    results["bertscore_precision"] = np.mean(bertscore_result["precision"])
    results["bertscore_recall"] = np.mean(bertscore_result["recall"])
    results["bertscore_f1"] = np.mean(bertscore_result["f1"])
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    results["gen_len"] = np.mean(prediction_lens)
    return results

# 5. TRAINING ARGUMENTS
training_args = Seq2SeqTrainingArguments(
    output_dir="./v3",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    logging_strategy="epoch",
    logging_dir="./v3",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust based on GPU
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=50,
    predict_with_generate=True,
    fp16=True,  # Enable if GPU supports
    report_to=["tensorboard"],
    warmup_steps=500,
    lr_scheduler_type="cosine",
)

# 6. TRAINER SETUP
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 7. TRAIN
trainer.train()
print('training completed')


Loaded 126160 records from training_data.json
Example record: {'question': 'Where is airq5.01 located?', 'entity': 'bldg:airq5.01', 'sparql': 'SELECT ?location WHERE { bldg:airq5.01 brick:hasLocation ?location . }'}
Generated 125110 total training pairs from 126160 records.
Train size: 112599 | Validation size: 12511
Saved train_data_April.json and val_data_April.json!
Train sample: {'input_text': 'task: generate_sparql\ninput: Show me the sensor name for Illuminance Sensor 5.03.\nentitybldg:Illuminance_Sensor_5.03', 'target_text': 'SELECT ?label WHERE { bldg:Illuminance_Sensor_5.03 rdfs:label ?label . }'}
Validation sample: {'input_text': 'task: generate_sparql\ninput: Tell me the name or label of the heating supply air temperature deadband setpoint in the Storage Room.\nentitybldg:Storage_Room \n brick:Heating_Supply_Air_Temperature_Deadband_Setpoint', 'target_text': 'SELECT ?label WHERE { ?sensor a brick:Heating_Supply_Air_Temperature_Deadband_Setpoint ; brick:hasLocation bldg:Stora

Map:   0%|          | 0/112599 [00:00<?, ? examples/s]

Map:   0%|          | 0/12511 [00:00<?, ? examples/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Meteor,Bertscore Precision,Bertscore Recall,Bertscore F1,Gen Len
1,0.000000,0.000001,0.474518,0.428388,0.474604,0.474564,0.090352,0.302615,0.917822,0.829124,0.871165,19.000000
2,0.000000,0.000001,0.474526,0.428388,0.474606,0.474565,0.090354,0.302615,0.917820,0.829125,0.871165,19.000000
3,0.000000,0.000000,0.474518,0.428388,0.474604,0.474564,0.090352,0.302615,0.917822,0.829124,0.871165,19.000000
4,0.000000,0.000000,0.474518,0.428388,0.474604,0.474564,0.090352,0.302615,0.917822,0.829124,0.871165,19.000000


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to 

KeyboardInterrupt: 

### Evaluation code

In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load your trained model and tokenizer from the saved directory.
model_dir = "./v3/checkpoint-56300"  # Update this path if needed
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_sparql(question, entity):
    """
    Generate a SPARQL query given a natural language question and an entity.
    """
    # Format the input as it was during training.
    input_text = f"task: generate_sparql\ninput: {question}\nentity{entity}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    
    # Generate output using beam search.
    outputs = model.generate(input_ids, max_length=150, num_beams=5, early_stopping=True)
    generated_sparql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_sparql

# Example input data
input_data = {
    "question": "Provide the definition for Heating Ventilation Air Conditioning System.",
    "entity": "brick:Heating_Ventilation_Air_Conditioning_System",
    "sparql": "SELECT ?definition WHERE { brick:Heating_Ventilation_Air_Conditioning_System skos:definition ?definition . }"
}

# Get outputs for both tasks
sparql_output = generate_sparql(input_data["question"], input_data["entity"])

print("Generated SPARQL Query:")
print(sparql_output)


Generated SPARQL Query:
SELECT ?definition WHERE { brick:Heating_Ventilation_Air_Conditioning_System skos:definition ?definition . }


### test on new data 

In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json

# Load your trained model and tokenizer from the saved directory.
# model_dir = "./v3/checkpoint-56300"  # Update this path if needed
# tokenizer = T5Tokenizer.from_pretrained(model_dir)
# model = T5ForConditionalGeneration.from_pretrained(model_dir)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

def generate_sparql(question, entity):
    """
    Generate a SPARQL query given a natural language question and an entity.
    """
    # Format the input as it was during training.
    input_text = f"task: generate_sparql\ninput: {question}\nentity: {entity}"  # Added colon after "entity" for consistency
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    
    # Generate output using beam search.
    outputs = model.generate(input_ids, max_length=150, num_beams=5, early_stopping=True)
    generated_sparql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_sparql

# JSON data (you can also load this from a file using json.load())
test_data = [
    {
        "question": "What is the average cooling command reading for AHU 01 over the last hour?",
        "entity": "bldg:bldg1.AHU.AHU01.CCV",
        "sparql": "SELECT ?timeseriesId WHERE { bldg:bldg1.AHU.AHU01.CCV ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId . }"
    },
    {
        "question": "Show me the most recent cooling command value for AHU 01.",
        "entity": "bldg:bldg1.AHU.AHU01.CCV",
        "sparql": "SELECT ?timeseriesId WHERE { bldg:bldg1.AHU.AHU01.CCV ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId . }"
    },
    {
        "question": "What is the current cooling command status of AHU 01?",
        "entity": "bldg:bldg1.AHU.AHU01.CCV",
        "sparql": "SELECT ?timeseriesId WHERE { bldg:bldg1.AHU.AHU01.CCV ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId . }"
    }
]

# Test the model with the JSON data
for i, item in enumerate(test_data, 1):
    question = item["question"]
    entity = item["entity"]
    original_sparql = item["sparql"]
    
    # Generate SPARQL query
    generated_sparql = generate_sparql(question, entity)
    
    # Print results
    print(f"\nTest Case {i}:")
#     print(f"Question: {question}")
    print(f"Entity: {entity}")
    print("Original SPARQL Query:")
    print(original_sparql)
    print("Generated SPARQL Query:")
    print(generated_sparql)
    print("-" * 80)


Test Case 1:
Entity: bldg:bldg1.AHU.AHU01.CCV
Original SPARQL Query:
SELECT ?timeseriesId WHERE { bldg:bldg1.AHU.AHU01.CCV ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId . }
Generated SPARQL Query:
SELECT ?timeseriesId ?storedAt WHERE { bldg:bldg1.AHU.AHU01.CCV ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId ; ref:storedAt ?storedAt . }
--------------------------------------------------------------------------------

Test Case 2:
Entity: bldg:bldg1.AHU.AHU01.CCV
Original SPARQL Query:
SELECT ?timeseriesId WHERE { bldg:bldg1.AHU.AHU01.CCV ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId . }
Generated SPARQL Query:
SELECT ?timeseriesId WHERE { bldg:bldg1.AHU.AHU01.CCV ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId ; ref:storedAt ?storedAt . }
-------------------------------------------

In [13]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
import re

# Load your trained model and tokenizer from the saved directory.
# model_dir = "./v3/checkpoint-56300"  # Update this path if needed
# tokenizer = T5Tokenizer.from_pretrained(model_dir)
# model = T5ForConditionalGeneration.from_pretrained(model_dir)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

def generate_sparql(question, entity):
    """
    Generate a SPARQL query given a natural language question and an entity.
    """
    # Format the input as it was during training.
    input_text = f"task: generate_sparql\ninput: {question}\nentity: {entity}"  # Added colon after "entity" for consistency
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    
    # Generate output using beam search.
    outputs = model.generate(input_ids, max_length=150, num_beams=5, early_stopping=True)
    generated_sparql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_sparql

# JSON data (you can also load this from a file using json.load())
test_data = [
    {
        "question": "How is sensor measuring temperature at the moment?",
        "entity": "bldg:AirTemperatureSensor_5.45"
    },
    {
        "question": "Retrieve the latest measurement from Alcohol Vapor MQ3 Gas Sensor.",
        "entity": "bldg:AlcoholVaporMQ3GasSensor_5.55 "
    },
    {
        "question": "Could you run an analysis on sensor drift for  PM1 particulate matter Level Sensor over time?",
        "entity": "bldg:PM1_LevelSensorAtmospheric_5.425"
    }
]

# Test the model with the JSON data
for i, item in enumerate(test_data, 1):
    question = item["question"]
    entity = item["entity"]
    
    # Generate SPARQL query
    generated_sparql = generate_sparql(question, entity)
    

    def add_space_before_question_mark(text):
        return re.sub(r'(?<!\s)\?', ' ?', text)

    generated_sparql = add_space_before_question_mark(generated_sparql)

    print("Generated SPARQL Query:")
    print(generated_sparql)
    print("-" * 80)

Generated SPARQL Query:
SELECT ?timeseriesId ?storedAt WHERE { bldg:AirTemperatureSensor_5.45 ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId ; ref:storedAt ?storedAt . }
--------------------------------------------------------------------------------
Generated SPARQL Query:
SELECT ?timeseriesId ?storedAt WHERE { bldg:AlcoholVaporMQ3GasSensor_5.55 ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId ; ref:storedAt ?storedAt . }
--------------------------------------------------------------------------------
Generated SPARQL Query:
SELECT ?timeseriesId ?storedAt WHERE { bldg:PM1_LevelSensorAtmospheric_5.425 ref:hasExternalReference ?ref . ?ref a ref:TimeseriesReference ; ref:hasTimeseriesId ?timeseriesId ; ref:storedAt ?storedAt . }
--------------------------------------------------------------------------------


In [14]:
import json
import random
import torch
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
import evaluate
from sklearn.metrics import precision_score, recall_score, f1_score

# Load your trained model and tokenizer
model_dir = "./v3/checkpoint-56300"  # Update this path if needed
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded on {device}")

# Load the sample.json file
json_file = "training_data.json"  # Update with your file path
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"Loaded {len(data)} records from {json_file}")

# Filter entries with required keys and select 50 random samples
required_keys = {"question", "entity", "sparql"}
filtered_data = [entry for entry in data if all(key in entry for key in required_keys)]
if len(filtered_data) < 50:
    raise ValueError(f"Insufficient entries with required keys. Found {len(filtered_data)}, need 50.")
sample_data = random.sample(filtered_data, 500)
print(f"Selected 500 random samples for evaluation.")

# Function to generate SPARQL query
def generate_sparql(question, entity):
    input_text = f"task: generate_sparql\ninput: {question}\nentity{entity}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = model.generate(input_ids, max_length=150, num_beams=5, early_stopping=True)
    generated_sparql = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return generated_sparql

# Generate predictions for all 50 samples
predictions = []
references = []
for entry in sample_data:
    pred = generate_sparql(entry["question"], entry["entity"])
    predictions.append(pred)
    references.append(entry["sparql"].strip())

# Load evaluation metrics
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")
meteor_metric = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

# Compute token-level Precision, Recall, F1 (treating SPARQL as sequences of tokens)
def compute_token_prf(predictions, references):
    pred_tokens = [pred.split() for pred in predictions]
    ref_tokens = [ref.split() for ref in references]
    
    # Flatten and binarize for token-level comparison
    all_pred_tokens = set([token for sublist in pred_tokens for token in sublist])
    all_ref_tokens = set([token for sublist in ref_tokens for token in sublist])
    
    # Create binary labels for each unique token
    y_true = [1 if token in all_ref_tokens else 0 for token in all_pred_tokens]
    y_pred = [1] * len(all_pred_tokens)  # Predicted tokens are all "positive" in this context
    
    # Handle edge case where no tokens overlap
    if not y_true or not y_pred:
        return 0.0, 0.0, 0.0
    
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return precision, recall, f1

# Compute all metrics
def compute_metrics(predictions, references):
    results = {}
    
    # ROUGE
    rouge_results = rouge_metric.compute(predictions=predictions, references=references, use_stemmer=True)
    results.update(rouge_results)
    
    # BLEU
    bleu_results = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
    results["bleu"] = bleu_results["bleu"]
    
    # METEOR
    meteor_results = meteor_metric.compute(predictions=predictions, references=references)
    results["meteor"] = meteor_results["meteor"]
    
    # BERTScore
    bertscore_results = bertscore_metric.compute(predictions=predictions, references=references, lang="en")
    results["bertscore_precision"] = np.mean(bertscore_results["precision"])
    results["bertscore_recall"] = np.mean(bertscore_results["recall"])
    results["bertscore_f1"] = np.mean(bertscore_results["f1"])
    
    # Token-level Precision, Recall, F1
    precision, recall, f1 = compute_token_prf(predictions, references)
    results["token_precision"] = precision
    results["token_recall"] = recall
    results["token_f1"] = f1
    
    # Average generation length
    results["gen_len"] = np.mean([len(pred.split()) for pred in predictions])
    
    return results

# Compute metrics
metrics = compute_metrics(predictions, references)

# Display results
print("\n=== Evaluation Results for 50 Random Samples ===")
print("\nSample Outputs (Original vs T5-Generated):")
for i, (ref, pred) in enumerate(zip(references[:5], predictions[:5]), 1):  # Show first 5 for brevity
    print(f"\nSample {i}:")
    print(f"Original SPARQL:  {ref}")
    print(f"T5-Generated:     {pred}")
    print("-" * 80)

print("\nEvaluation Metrics:")
for metric, value in metrics.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

# Clear CUDA cache (optional)
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("\nCUDA cache cleared.")

Model loaded on cuda
Loaded 126160 records from training_data.json
Selected 50 random samples for evaluation.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Evaluation Results for 50 Random Samples ===

Sample Outputs (Original vs T5-Generated):

Sample 1:
Original SPARQL:  SELECT ?label WHERE { ?sensor a brick:Wind_Speed_Sensor ; brick:hasLocation bldg:Waiting_Area ; rdfs:label ?label . }
T5-Generated:     SELECT ?label WHERE { ?sensor a brick:Wind_Speed_Sensor ; brick:hasLocation bldg:Waiting_Area ; rdfs:label ?label . }
--------------------------------------------------------------------------------

Sample 2:
Original SPARQL:  SELECT ?label WHERE { ?sensor a brick:Radiant_Panel_Temperature_Setpoint ; brick:hasLocation bldg:Pantry ; rdfs:label ?label . }
T5-Generated:     SELECT ?label WHERE { ?sensor a brick:Radiant_Panel_Temperature_Setpoint ; brick:hasLocation bldg:Pantry ; rdfs:label ?label . }
--------------------------------------------------------------------------------

Sample 3:
Original SPARQL:  SELECT ?location WHERE { bldg:TVOC_Level_Sensor_5.04 brick:hasLocation ?location . }
T5-Generated:     SELECT ?location WHERE {